Two Dimensional Parity Bits


In [1]:
import numpy as np
from random import randrange

In [2]:
def eliminate_zeros(input_list: list)->list:
    '''
    Eliminates all the zeros found at the beginning of the list until it finds a non-zero element.
    Returns the modified list.
    '''
    while input_list and input_list[0] == 0:
        input_list.pop(0)
    return input_list


def check_binary(input_list: list)->bool:
    '''
    Checks the given list if it only contains binary characters (0 and 1).
    Returns false if the criteria is not met.
    '''
    for element in input_list: 
        if element not in list([0,1]):
            print("Your string must contain only binary characters.")
            return False
    return True


def check_length(input_list: list)->bool:
    '''
    Checks the given list if the length of the string is a multiple of seven.
    Returns false if the criteria is not met.
    '''
    if len(input_list) % 7 != 0:
        print("The length of your string must be a multiple of seven.")
        return False
    return True


def process_sums(input_matrix: np.array, by_row: bool = True)->np.array:
    '''
    1. Takes every line column or row-wise and counts the number of 1 found on each, without the last element. 
        The last element of the line is given the value:
            - 0 if there is an even number of one,
            - 1 if there is an odd number of one.
    2. Calculates the binary addition of elements with value of 1 on every line, row or column-wise.
    
    The default calculations will be made row-wise. For column-wise calculations, the second argument should be False.
    
    Returns the modified array.
    '''
    if not by_row:
        input_matrix = input_matrix.T
        
    for line in input_matrix:
        sum_of_ones = np.count_nonzero(line)
        if sum_of_ones % 2 == 0:
            line[-1] = 0
        else:
            line[-1] = 1
    return input_matrix


def convert_and_process_matrix(input_list: list)->np.array:
    '''
    Converts the given list to a NumPy array of shape (N+1, 8) and calculates the sum of one on each column and line.
    N is the quotient of the division of the string length to 7.
    
    Returns a matrix.
    '''
    matrix = np.array(input_list).reshape(len(input_list) // 7, 7)
    matrix = np.append(matrix, np.zeros((len(input_list) // 7, 1), dtype = matrix.dtype), axis = 1)
    matrix = np.append(matrix, np.zeros((1,8), dtype = matrix.dtype), axis = 0)
    
    matrix = process_sums(matrix)
    matrix = process_sums(matrix, False).T
    
    return matrix
 
    
def simulate_error(input_matrix: np.array)->np.array:
    '''
    Simulates the appearance of an error by choosing a random position in the matrix and changing its value.
    
    Returns the modified matrix.
    '''
    
    error_list = list(input_matrix.flatten())
    error_index = randrange(len(error_list))
    print("Error index: ", error_index)
    error_list[error_index] = 0 if error_list[error_index] == 1 else 1
    return np.array(error_list).reshape(input_matrix.shape)


def return_error_index(input_matrix: np.array, by_row: bool=True)->int:
    '''
    Returns the index of the column or row where the sum does not match with its addends.
    The default calculations will be made row-wise. For column-wise calculations, the second argument should be False.
    '''
    
    if not by_row:
        input_matrix = input_matrix.T
    
    for index, line in enumerate(input_matrix):
        sum_of_ones = np.count_nonzero(line[:-1])
        sum_of_ones = 0 if sum_of_ones % 2 == 0 else 1
        if line[-1] != sum_of_ones:
            return index
    return -1


def find_error(input_matrix: np.array):
    '''
    Prints the exact position of the flipped bit.
    '''
    
    row_index = return_error_index(input_matrix)
    column_index = return_error_index(input_matrix, False)
    print("Error was found at position: (",row_index,", ",column_index, sep="", end=").")

In [3]:
input_string = input("Enter the string: ") 
try:
    bit_list = list(map(int,input_string)) 
    
    bit_list = eliminate_zeros(bit_list)
    print(bit_list)
    if check_binary(bit_list) and check_length(bit_list):
        print("Initial list:\n", bit_list)
        
        bit_matrix = convert_and_process_matrix(bit_list)
        print("Initial matrix:\n", bit_matrix)
        
        bit_matrix = simulate_error(bit_matrix)
        print("Error matrix:\n", bit_matrix)
        
        find_error(bit_matrix)
except ValueError:
    print("Make sure your string only contains numbers!")

Enter the string:  1001011


[1, 0, 0, 1, 0, 1, 1]
Initial list:
 [1, 0, 0, 1, 0, 1, 1]
Initial matrix:
 [[1 0 0 1 0 1 1 0]
 [1 0 0 1 0 1 1 0]]
Error index:  11
Error matrix:
 [[1 0 0 1 0 1 1 0]
 [1 0 0 0 0 1 1 0]]
Error was found at position: (1, 3).

Cyclic Redundancy Check

In [4]:
def check_message_length(input_string_list: list, input_poly_list: list) -> bool:
    if len(input_string_list) < len(input_poly_list):
        return False
    return True

def add_zeros_at_end(input_string_list: list, input_poly_list: list) -> list:
    num_zeros = len(input_poly_list) - 1
    input_string_list.extend([0] * num_zeros)
    return input_string_list

def add_result_to_message(input_list: list, input_result: list)->list:
    for i in range(1,len(input_result) + 1):
        input_list[-i] = input_result[-i]
    return input_list

def XOR_operation(input_string_list: list, input_poly_list: list)->list:
    step_index = 0
    while len(input_string_list) >= len(input_poly_list):
        i = 0
        while i < len(input_poly_list):
            input_string_list[i] = input_string_list[i] ^ input_poly_list[i]
            i += 1
        eliminate_zeros(input_string_list)
        print("[", step_index, "] :", ''.join(str(e) for e in input_string_list))
        step_index += 1
    return input_string_list


In [5]:
crc_input_string = input("Enter the message: ")
generating_polynomial = input("Enter the generating polynomial: ")

#10010101110101
#1011

#10101110101110
#1011

try:
    crc_input_list = list(map(int, crc_input_string))
    generating_polynomial_list = list(map(int, generating_polynomial))
    
    crc_input_list = eliminate_zeros(crc_input_list)
    generating_polynomial_list = eliminate_zeros(generating_polynomial_list)
    
    if check_message_length(crc_input_list, generating_polynomial_list):
        crc_input_list = add_zeros_at_end(crc_input_list, generating_polynomial_list)
        print("Extended Message:",crc_input_list)
        crc_list2=[e for e in crc_input_list]
        xor_result = XOR_operation(crc_input_list, generating_polynomial_list)
        print("Result of first XOR:", xor_result)
        crc_list2=add_result_to_message(crc_list2, xor_result)
        print("Transmitted message:",crc_list2)
        
        
        message = XOR_operation(crc_list2, generating_polynomial_list)
        print("Result of the XOR operation between transmitted message and generating polynomial:", message)
        if(message):
            print("Message was transmitted with errors!")
        else:
            print("Message was transmitted successfully!")
            
except ValueError:
    print("Make sure your strings only contain numbers!")

Enter the message:  10101110101110
Enter the generating polynomial:  1011


Extended Message: [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0]
[ 0 ] : 11110101110000
[ 1 ] : 1000101110000
[ 2 ] : 11101110000
[ 3 ] : 1011110000
[ 4 ] : 110000
[ 5 ] : 11100
[ 6 ] : 1010
[ 7 ] : 1
Result of first XOR: [1]
Transmitted message: [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1]
[ 0 ] : 11110101110001
[ 1 ] : 1000101110001
[ 2 ] : 11101110001
[ 3 ] : 1011110001
[ 4 ] : 110001
[ 5 ] : 11101
[ 6 ] : 1011
[ 7 ] : 
Result of the XOR operation between transmitted message and generating polynomial: []
Message was transmitted successfully!
